In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.datasets import mnist

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [3]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

Y_train = tf.keras.utils.to_categorical(Y_train)
Y_test = tf.keras.utils.to_categorical(Y_test)

In [4]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
conv1_k = 5
stride_conv1 = 1
max_pool1_k = 2

n_conv2 = 64
conv2_k = 5
stride_conv2 = 1
max_pool2_k = 2

n_hidden = 1024
n_out = 10

In [5]:
weights = {
    'wc1' : tf.Variable(tf.random.normal([conv1_k, conv1_k, input_channels, n_conv1])),
    'wc2' : tf.Variable(tf.random.normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    'wh1' : tf.Variable(tf.random.normal([(input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*(n_conv2), n_hidden])),
    'wo' : tf.Variable(tf.random.normal([n_hidden, n_out]))
}

biases = {
    'bc1' : tf.Variable(tf.random.normal([n_conv1])),
    'bc2' : tf.Variable(tf.random.normal([n_conv2])),
    'bh1' : tf.Variable(tf.random.normal([n_hidden])),
    'bo' : tf.Variable(tf.random.normal([n_out]))
}

In [6]:
def convolution(X, weights, biases, stride = 1):
  output = tf.nn.conv2d(X, weights, strides = [1, stride, stride, 1], padding = 'SAME')
  output = tf.nn.bias_add(output, biases)
  output = tf.nn.relu(output)
  return output 

def maxpooling(X, k = 2):
  return tf.nn.max_pool(X, ksize = [1, k, k, 1], strides = [1, k, k, 1], padding = 'SAME')

In [7]:
def cnn(X, weights, biases, keep_prob):
  tf.reshape(X, shape = [-1, input_width, input_height, input_channels])
  
  conv1 = convolution(X, weights['wc1'], biases['bc1'], stride_conv1)
  max_pool1 = maxpooling(conv1, max_pool1_k)
  
  conv2 = convolution(max_pool1, weights['wc2'], biases['bc2'], stride_conv2)
  max_pool2 = maxpooling(conv2, max_pool2_k)

  hidden_input = tf.reshape(max_pool2, shape = [-1, (input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*(n_conv2)])
  hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
  hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
  hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob = keep_prob)

  output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
  return output

In [8]:
X = tf.placeholder('float', [None, input_width, input_height, input_channels])
Y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder('float')
predictions = cnn(X, weights, biases, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = predictions, labels = Y))

In [10]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [11]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [12]:
batch_size = 100
for i in range(30):
    num_batches = X_train.shape[0]//batch_size
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = X_train[(batch_size)*((j-1) + 1):(batch_size)*(j + 1)], Y_train[(batch_size)*((j-1) + 1):(batch_size)*(j + 1)]
        c, _ = session.run([cost,optimize], feed_dict = {X : batch_x , Y : batch_y, keep_prob : 0.8})
        total_cost += c
    print(total_cost)

210761297.70428467
9602551.91255188
4775957.81886673
2820271.2697734833
2298819.2201566696
2302541.9937057495
1882851.6473255157
1936090.3348546028
1581299.3637580872
1652156.0100933313
1231205.5764341354
1181665.0497956276
1025366.9485554695
1302603.026834488
868921.4429721832
1017658.8087863922
687028.8094615936
884557.0963525772
765961.9762438536
608645.4212729931
706923.1929469109
1012878.7100073099
559260.2942724228
648561.8086211681
539255.1690759659
523004.439661026
551939.8937356472
439275.47355890274
530294.6708650589
725277.221704483


In [13]:
predictions = tf.argmax(predictions, 1)
correct_labels = tf.argmax(Y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions, correct_preds  = session.run([predictions, correct_predictions], feed_dict={X : X_test, Y : Y_test, keep_prob : 1.0})
print(f'Accuracy: {(correct_preds.sum()/len(correct_preds))*100}%') 

Accuracy: 98.79%
